# Problem definition
A big problem in every software development project is to determine a set of requirements which satisfies all parts involved (stakeholders). The next release problem (NRP) provides a formal mathematical model to this problem. This problems aims to find a subset of requirements or stakeholders that optimize a wanted attribute, such as profit or cost.
NRP can be reduced to the knapsack problem which, in the end, is a NP-Hard problem.

## Formal definition of NRP Mono-Objective

The problem can be addressed as a mixed integer linear programming (MILP) model. The next model assumes that the decision maker wants to maximize the profit subject to a cost restriction. 

- Let $S$ be the set of stakeholders with $|S| = n$
- Let $R$ be the set of requierements with $|R| = m$
- Let $ p $ be the max cost affordable.
- Let $ X = [x_1 , x_2 , … ,x_n ] $ be a binary array where the value of the cell $ i = 1$ if $i$ requirement if implemented. $ 0 $ otherwise 
- Let $ Y = [y_1, y_2,…,y_m] $ be a binary array where the value of the cell $ j = 1$ if the $j$ stakeholder is satisfied (that means, all of his requirement are implemented in the next release). $0$ otherwise.
- Let $ C = [c_1, c_2, …, c_n ] $ be an array of cost per requirement.
- Let $B = [b_1, b_2 ,… ,b_m] $ the profit of satisfy a stakeholder
- Let $P$ be the precedence relation between $(i,j)$ where $i,j$ are requirements; meaning that $i$ requirement must be selected if $j$ requirement is selected.
- Let $ I $ be the interest relation $(i,k)$ where $k$ stakeholder has interest over $i$ requierement.

With this parameters, the model looks like:

$$
max \ f(Y) = \sum_{i \in S} b_i \cdot y_i
$$
subject to:

1) A cost restriction. The cost of implementing each requierement needs to be less than the max cost affordable
$$
\sum_{j \in R} c_j \cdot x_j \leq p
$$

2) A precedence restriction. If requierement $x_j$ needs to be implemented (i.e. $x_j = 1$) $x_i$ must be equal to 1 in order to not violate this restriction
$$
x_i \geq x_j \quad \forall (i,j) \in P
$$

3) An interest constraint. This restriction is used to set $y_k$ if requierement $x_i$ is implemented. Since the objetive funcition is a maximization, if $x_i$ is implemented, $y_k$ is automatically  set to 1 because it maximize the objetive.
$$
x_i \geq y_k \quad \forall (i,k) \in I
$$

4) Binary constraints
$$
X \in \{0,1\}^n
$$
$$
Y \in \{0,1\}^n
$$

# Implementing NRP in python
We'll be using python to find an optimal solution of this problem. 
Why python you may ask. The answer is python has an excellent collection of libraries to model the problem. Moreover, these libraries are capable of calling  low level solvers such as CBC or CPLEX to solve the problem.

We are renaming the parameters in order to achieve a more readeable code.

First, let's begin with importing the needing libraries

In [1]:
import pyomo.environ as pyo
from pyomo.environ import AbstractModel

[Pyomo](http://www.pyomo.org/) is a library that provide common language for modelling and is able to call different solvers.

In [4]:
# Create and abstract model
nrp_abs = pyo.AbstractModel()

# Assign parameters to the model
nrp_abs.number_of_requirements = pyo.Param(within=pyo.NonNegativeIntegers)
nrp_abs.number_of_stakeholders = pyo.Param(within=pyo.NonNegativeIntegers)
nrp_abs.max_cost = pyo.Param(within=pyo.NonNegativeIntegers, mutable=True)

# Sets used to maintain data of customers and requeriments
nrp_abs.requirements = pyo.RangeSet(1, nrp_abs.number_of_requirements)
nrp_abs.stakeholders = pyo.RangeSet(1, nrp_abs.number_of_stakeholders)


# Parameters for the model
nrp_abs.profit = pyo.Param(nrp_abs.stakeholders)  # Profit of each customer if it is satisfied
nrp_abs.cost = pyo.Param(nrp_abs.requirements)  # Cost of implementing each requierement


# (i,j) requierement i should be implemented if j is implemented
# Set is within the cross product of Requierements X Requierements
nrp_abs.prerequisite = pyo.Set(within=nrp_abs.requirements * nrp_abs.requirements)
# (i,k) this relation exists if stakeholder k has interest on requierement i
nrp_abs.interest = pyo.Set(within=nrp_abs.stakeholders * nrp_abs.requirements)

# Creation of variables
# x = 1 if requierement i is implemented in the next release, otherwise 0
nrp_abs.x = pyo.Var(nrp_abs.requirements, domain=pyo.Binary)
# y = 1 if all customer requierements are satisfied in the next release, otherwise 0
nrp_abs.y = pyo.Var(nrp_abs.stakeholders, domain=pyo.Binary)

# Objetive function
def obj_expression(nrp: AbstractModel):
    # Model should maximize profit of the next release
    return pyo.summation(nrp.profit, nrp.y)
nrp_abs.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)


# Defintion of cost constraint rule
def cost_constraint_rule(nrp: AbstractModel):
    # Cost should be maintened under a predefined cost
    return pyo.summation(nrp.cost, nrp.x) <= nrp.max_cost
nrp_abs.cost_constraint = pyo.Constraint(rule=cost_constraint_rule)


# Defition of precedence constraint
def precedence_constraint_rule(nrp: AbstractModel, i: int, j: int):
    return nrp.x[i] >= nrp.x[j]
nrp_abs.precedence_constraint = pyo.Constraint(nrp_abs.prerequisite, rule=precedence_constraint_rule)

# Definition of interest constraint
# Each tuple in nrp.dat.interest is inverted, so the constraint is also inverted
def interest_constraint_rule(nrp: AbstractModel, i: int, k: int):
    return nrp.y[i] <= nrp.x[k]
nrp_abs.interest_constraint = pyo.Constraint(nrp_abs.interest, rule=interest_constraint_rule)

Now, we have an abstract model. This is very powerful since now, we can provide differnt dataset to see how the model performs.

Lets fill the model with actual data and solve it.

In [31]:
data_file_path = './datasets/data.dat'
nrp_concrete = nrp_abs.create_instance(data_file_path)

## Solving the model
We are using [CBC](https://github.com/coin-or/Cbc) beacuste it's an open source and fast, but if you desire you can use different solvers.

In [32]:
from pyomo.environ import SolverFactory

nrp_concrete.max_cost = 80


solver = SolverFactory('cbc')

res = solver.solve(nrp_concrete)
res.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 70.0
  Upper bound: 70.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 6
  Number of binary variables: 7
  Number of integer variables: 7
  Number of nonzeros: 2
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      

## Exploring the solution
We can see the whole model using `pyo.display(nrp_concrete)` or we can use `pyo.display` on the parts  where we have interest

In [12]:
# Print the value of the objetive function
pyo.display(nrp_concrete.OBJ)

OBJ : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True :  70.0


In [13]:
# Dipslay of variables asociated to requierements
# Value == 1 means the requieremnt is implemented
pyo.display(nrp_concrete.x)

x : Size=5, Index=requirements
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   1.0 :     1 : False : False : Binary


In [14]:
# Print information about cost constraint
# In this case, we are in the upper limit of the constraint
pyo.display(nrp_concrete.cost_constraint)

cost_constraint : Size=1
    Key  : Lower : Body : Upper
    None :  None : 80.0 :    80


In [15]:
# We can print the state of the precedence constraint
# If body == -1 it means that the only the precedende requierement is implemented
# If body == 0 it means that none or both of the requierements are implemented
pyo.display(nrp_concrete.precedence_constraint)

precedence_constraint : Size=2
    Key    : Lower : Body : Upper
    (1, 3) :  None : -1.0 :   0.0
    (1, 5) :  None :  0.0 :   0.0


In [29]:
# To see which stakeholders are satisfied
# If value == 1 it is satisfied
pyo.display(nrp_concrete.y)

y : Size=2, Index=stakeholders
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary


## Using pyomo features
Using pyomo brings up a lot of features like changing parameter's model once created. We can see different different results with only a few lines of code. 

In [83]:
nrp_concrete.max_cost = 200
res = solver.solve(nrp_concrete)
pyo.display(nrp_concrete)

Model unknown

  Variables:
    x : Size=140, Index=requierements
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   0.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary
         11 :     0 :   1.0 :     1 : False : False : Binary
         12 :     0 :   0.0 :     1 : False : False : Binary
         13 :     0 :   1.0 :     1 : False : False : Binary
         14 :     0 :   0.0 :     1 : False : False : Binary
         15 :     0

# Solving time details

We have solved a quite small model, with a few variables and constraints. The model was resolved in 0.2 sec which is quite fast. What will happen if the model is bigger? We'll se 

In [12]:
data_file_path = '../datasets/nrp_750c_3250r.dat'
nrp_heavy = nrp_abs.create_instance(data_file_path)

nrp_heavy.max_cost = 2000

res = solver.solve(nrp_heavy)
res.write()

    taken
    taken
    taken
    taken
    taken
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4750.0
  Upper bound: 4750.0
  Number of objectives: 1
  Number of constraints: 5964
  Number of variables: 3325
  Number of binary variables: 4000
  Number of integer variables: 4000
  Number of nonzeros: 750
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 11.1
  Wallclock time: 11.36
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an opti

11 second to solve a model with almost 6000 constraints is quite impressive. But we'll in the following posts that some models requiere more time to solve so we need to do some tricks to overcome this.